# Time domain (TD) experiments

This notebook will detail the flow of the time domain (TD) experiments used to characterize the shelving fluxonium qubit.


In [4]:
from plottr.data.datadict_storage import datadict_from_hdf5
from plottr.data.datadict_storage import DataDict, DDH5Writer
from plottr.data.datadict_storage import search_datadict
import numpy as np
import matplotlib.pyplot as plt
import os
from pprint import pprint
from tqdm import tqdm
from datetime import datetime
import pandas as pd
import copy
import itertools
# import scqubits as scq
import csv
from sklearn.decomposition import PCA
from scipy.signal import find_peaks
from lmfit import Model
from scipy.optimize import curve_fit

Like in CW, our search function:

In [5]:
basedir = r"D:/Redza/Logs"

#Fido function to obtain monitr files
def Fido(datetime, name):          #Fido gets your raw _data for you!
    foldername, datadict = search_datadict(basedir, datetime, name=name , newest = True, only_complete=False)
    print(foldername)
    print(datadict)
    return foldername, datadict

Before actually characterizing the qubit, each of the TD drive have to be calibrated. This is because a TD signal is produced by mixing an LO output, which provides the carrier frequency, and outputs from AWGs, which are combined in mixers which are either (at least to my knowledge), single sideband (SSB), requiring a single IF input, or IQ, requiring 2. The SSB mixer calibration is more simply just optimization; simply ensure the output signal frequencies are well spaced in frequency and power. IQ calibration requires a little more, and the code iq_mixer_calibrate.py and iq_mixer_check.py are how we calibrate the IQ-mixed signals.

After all of the drives have been properly calibrated based on their mixers, we want to then ensure that the calibrations work. For the readout drive itself, we can simply test to see that the drive is being shaped properly. The code td_pulse_check.py emits and directly reads the waveform of the readout drive. You can customize it as you want, but seeing a square pulse in the resulting pop-up window should be enough for confirmation. This also obviously depends on the attenuation in the readout line; an output directly connected into the signal input gives a much clearer shape than one that has to go into the fridge, for example.

To ensure the JPA pulse properly amplifies the readout signal, while we could also just check the JPA line's pulse shaping, it is much better to check the vacuum squeezing cause by the JPA. First, we can do a simple confirmation test using the code in TD_20_JPA_timing.py. It plots the standard deviation and the mean of the measurement of the vacuum, which is squeezed as our JPA is pumped at a suitable value.

Next, (code) will squeeze the vacuum at different JPA pump powers to show where the pumping starts to cause parametric oscillations.

## Identifying readout saturation

This is still quite unreliable so far. All we can hope to do at the moment is use our information from CW measurements, namely the shelving frequency, and minimize the readout power. Results should show regardless.

## Identifying the qubit control frequency

This step is to essentially reconfirm (for the second time) the control frequency of the qubit. We drive the qubit with a square pulse, and sweep the frequency of the driving pulse. We then read out the qubit after. This detects qubit transitions that are directly related to the ground and shelving subspace of the qubit. In most cases, we are restricted to a very small sweep region, as we need to ensure that the mixing of the drive signal is maintained. However, in the case of a qubit frequency less than 1 GHz, we can directly drive the qubit using the AWG, which we do here.

In cases where the qubit frequency is not visible at all, the readout power may need to be decreased (reducing saturation), or maybe the drive pulse itself is too strong. In such a case, simultaneously sweeping both the drive amplitude and frequency can help, as we are more easily able to spot the points of drive saturation. 

Once we have nailed down the drive frequency, we perform a standard Rabi oscillation observation by sweeping the amplitude of a flat-top pulse at that frequency. This experiment should show Rabi oscillations, and depending on how many pulses are sent in, we can identify the pulse amplitude that corresponds to the optimal $\pi$-pulse frequency.

## Optimizing the JPA Phase

As stated before, the application of a JPA pulse causes the signal leaving the qubit to be amplified. This amplification is accompanied with a squeezing in the IQ plane, and there is thus an optimal squeezing axis where the 0 and 1 states of the qubit are maximally separated in the IQ plane. 

This should vastly improve the results in all experiments where the 0 and 1 states are read out from now on. 

## Identifying the photon emission frequency

The photon emission procedure is the main novelty in this research. Using a 2-photon process, we drive the qubit directly from 0 to 2, bypassing the usual fluxonium parity protection. The 2 state cannot decay back to 0 due to this protection, and instead decays to 1 via the shelving transition. This produces a (single) fluorescence photon, that we measure. 

While we can reliably predict the 0-2 transition frequency from our information on the qubit spectrum, it is better to try and observe this experimentally. We do another power and frequency sweep, this time around the 0-2 two photon expected frequency. Run the file td_2photongf_chevron_JPAPulsed.py, and observe the results. Not only should we be able to see a transition trace around the expected frequency, which shows up only beyond a certain power threshold, but we should also see some AC Stark shift, which is the gradual shift in the frequency of the 2-photon transition as the power increases.

Once the emission frequency and power is determined, then we can proceed to the photon emission experiments. Initially, set the 02 frequency to the frequency where the pulse amplitude is the lowest. This helps to ensure the AC Stark effect minimally affects the emission. This initial test helps give a feel for the photon emission time, and the emission pulse's length should be calibrated accordingly.

Once the photon waveform is properly observed (for a square wave, we should see a sharp initial rise, followed by an exponential decay), we may do more experiments to see the dependence of the waveform on the emission pulse frequency and amplitude.